In [2]:
import data_utils
import pandas as pd
import numpy as np

dataset = data_utils.get_dataset()
cie10_df = pd.read_csv('datasets/CIE10.csv', sep = ';')
cie10_df['DESCRIPCION_COD_CIE_10_04'] = cie10_df['DESCRIPCION_COD_CIE_10_04'].apply(lambda value: value.lower())
dataset = pd.merge(left = dataset, right = cie10_df, how = 'left', left_on='CIE_10', right_on='DESCRIPCION_COD_CIE_10_04')


In [3]:

def to_year(value_range):
    if(value_range == 'no_cie10'):
        return -1
    
    num = int(value_range[:3])
    unit = value_range[-1]
    
    map_unit = {
        'A': num,
        'M': num / 12,
        'D': num / 365,
        'H': num / 8760
    }
    return int(map_unit[unit])

def get_edad_inf(value):
    value_range = {
        '999': -1,
        'de 0 a 5 años': 0,
        'de 13 a 17 años': 13,
        'de 18 a 24 años': 18,
        'de 25 a 29 años': 25,
        'de 30 a 37 años': 30,
        'de 38 a 49 años': 38,
        'de 50 a 62 años': 50,
        'de 6 a 12 años': 6,
        'mayor de 63 años': 63
            }
    return value_range.get(value, -1)

def get_edad_sup(value):
    value_range = {
        '999': -1,
        'de 0 a 5 años': 5,
        'de 13 a 17 años': 17,
        'de 18 a 24 años': 24,
        'de 25 a 29 años': 29,
        'de 30 a 37 años': 37,
        'de 38 a 49 años': 49,
        'de 50 a 62 años': 62,
        'de 6 a 12 años': 12,
        'mayor de 63 años': 120
            }
    return value_range.get(value, -1)

def in_range(row):
    return row['AFEC_EDADR_INF'] >= row['LIMITE_INFERIOR_EDAD_Y'] and  row['AFEC_EDADR_SUP'] < row['LIMITE_SUPERIOR_EDAD_Y']

def cie10_sexo(value):
    if(value == 1 or value == 2):
        return True
    return False


In [4]:


dataset = dataset.drop(['CIE_10', 'NOMBRE_CAPITULO', 'DESCRIPCION_COD_CIE_10_03', 'DESCRIPCION_COD_CIE_10_04'], axis = 1)

cie10_columns = [
    'CAPITULO', 
    'COD_CIE_10_03', 
    'COD_CIE_10_04', 
    'SEXO', 
    'LIMITE_INFERIOR_EDAD', 
    'LIMITE_SUPERIOR_EDAD']

dataset[cie10_columns] = dataset[cie10_columns].replace(np.nan, 'no_cie10', regex=True)
dataset = dataset[dataset['CAPITULO'] != 'no_cie10']

dataset['CIE10_SEXO'] = dataset['SEXO'].apply(cie10_sexo)
dataset['LIMITE_INFERIOR_EDAD_Y'] = dataset['LIMITE_INFERIOR_EDAD'].apply(to_year)
dataset['LIMITE_SUPERIOR_EDAD_Y'] = dataset['LIMITE_SUPERIOR_EDAD'].apply(to_year)
dataset['AFEC_EDADR_INF'] = dataset['AFEC_EDADR'].apply(get_edad_inf)
dataset['AFEC_EDADR_SUP'] = dataset['AFEC_EDADR'].apply(get_edad_sup)
dataset['CIE10_RANGO_EDAD'] = dataset.apply(in_range, axis=1)

dataset = dataset.drop(
    [
        'SEXO',
        'LIMITE_INFERIOR_EDAD', 
        'LIMITE_SUPERIOR_EDAD', 
        'LIMITE_INFERIOR_EDAD_Y', 
        'LIMITE_SUPERIOR_EDAD_Y',
        'AFEC_EDADR'

    ], axis = 1
)


print(dataset.shape)
dataset[
    [
        'AFEC_GENERO',
        'CIE10_SEXO',
        'CIE10_RANGO_EDAD',
        'AFEC_EDADR_INF', 
        'AFEC_EDADR_SUP',

    ]
].head()


KeyboardInterrupt: 

In [ ]:
'''
import numpy as np
codes = dataset[['ENT_COD_DEPTO', 'ENT_MPIO']].drop_duplicates().to_dict('records')
codes
codes_depto = {}
for code in codes:
    codes_depto[code['ENT_MPIO']] = code['ENT_COD_DEPTO']
dataset['AFEC_COD_DPTO'] = dataset['AFEC_MPIO'].apply(lambda value: codes_depto.get(value, '999'))
dataset = dataset.drop(['AFEC_DPTO'], axis = 1)
'''

In [ ]:
dataset = data_utils.clean_afec_dpto(dataset)
dataset = data_utils.clean_riesgo_vida(dataset)
dataset = data_utils.remove_features(dataset)
dataset = dataset.reset_index()
dataset = dataset.drop(['index'], axis = 1)
print(dataset.shape)

In [ ]:
labels = dataset[['RIESGO_VIDA']]
features = dataset.drop(['RIESGO_VIDA'], axis = 1)

encoded_features = data_utils.encode_features(features, labels)

encoded_features['RIESGO_VIDA'] = labels

In [ ]:
def feature_correlations(df, feature):
    data_corr = df.corr()
    data_corr = data_corr.loc[feature].to_frame(name = 'correlation').reset_index()
    data_corr.plot(kind = 'bar', x = 'index', y = 'correlation', figsize=(15,5))
    print(data_corr[data_corr['index'] != feature]['correlation'].mean())

feature_correlations(encoded_features, 'RIESGO_VIDA')

In [ ]:
display(encoded_features.head(n = 5))

In [ ]:
encoded_features.to_csv("datasets/cie10_dataset.csv", index = False)

In [ ]:
values = {
    'key1': 'value1',
    'key2': 'value2',
    'key3': 'value3',
}

for key in values:
    print(key)